<a href="https://colab.research.google.com/github/gabriel1628/End-to-end-MLOps-for-Time-Series/blob/main/lgbm_hpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python version

In [1]:
import sys
sys.version

'3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]'

# If run in Google Colab

In [2]:
COLAB = True # if notebook is run in Google Colab

In [3]:
if COLAB:
    !pip install boto3 mlforecast optuna lightgbm GPUtil -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [4]:
if COLAB:
    import requests
    from pathlib import Path

    def download_file(url, output_path, headers=None):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded file: {output_path}")
        else:
            print(f"Failed to download file from {url}. Status code: {response.status_code}")

    def download_directory_contents(repo_owner, repo_name, directory, branch, output_path, headers=None):
        url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{directory}?ref={branch}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            contents = response.json()
            for item in contents:
                item_path = Path(directory, item['name'])
                output_item_path = output_path / item['name']

                if item['type'] == 'file':
                    download_file(item['download_url'], output_item_path, headers=headers)
                elif item['type'] == 'dir':
                    output_item_path.mkdir(parents=True, exist_ok=True)
                    download_directory_contents(repo_owner, repo_name, item_path, branch, output_item_path, headers=headers)
        else:
            print(f"Failed to fetch contents of directory '{directory}'. Status code: {response.status_code}")
            print(response.json())

    def download_directories(repo_owner, repo_name, directories, branch='main', output_path='.', token=None):
        headers = {}
        if token:
            headers['Authorization'] = f'token {token}'

        output_path = Path(output_path)
        output_path.mkdir(parents=True, exist_ok=True)

        for directory in directories:
            directory_output_path = output_path / directory
            directory_output_path.mkdir(parents=True, exist_ok=True)
            print(f"\nDownloading contents of directory: {directory}")
            download_directory_contents(repo_owner, repo_name, directory, branch, directory_output_path, headers=headers)
            # print(f"Contents of directory '{directory}' downloaded successfully to {directory_output_path}.\n")

    repo_owner = 'gabriel1628'
    repo_name = 'End-to-end-MLOps-for-Time-Series'
    directories = ['configuration_files', 'optuna_studies', 'preprocessing']
    files = ['utils.py']
    branch = 'main'

    # Optional: Use a GitHub token for authentication (recommended for private repositories)
    download_directories(repo_owner, repo_name, directories, branch)
    print()
    for file in files:
        url = f"https://raw.githubusercontent.com/{repo_owner}/{repo_name}/{branch}/{file}"
        download_file(url, output_path=Path(file), headers=None)


Downloaded file: configuration_files/lightgbm/config_1.yaml

Downloaded file: optuna_studies/lightgbm_preprocessing2_config1.db

Downloaded file: preprocessing/preprocessing.py
Downloaded file: preprocessing/preprocessing_test.ipynb

Downloaded file: utils.py


# Import libraries

In [5]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error

import logging
import optuna

import boto3

import GPUtil
import os
import sys
from pathlib import Path
import yaml
import time
from natsort import natsorted

from utils import train_test_split
from preprocessing.preprocessing import *

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Set device

In [6]:
device="gpu" if GPUtil.getAvailable() else "cpu"
print(f"device set to {device}")

device set to cpu


In [7]:
# import subprocess

# try:
#     subprocess.check_output('nvidia-smi')
#     device = "cuda"
# except Exception: # this command not being found can raise quite a few different errors depending on the configuration
#     device = "cpu"

# Global variables

In [8]:
forecast_horizon = 48
n_lags = 48
model_name = "lightgbm"
preprocessing_version = 2
config_version = None
config_dir_path = "./configuration_files"
random_state = 0

# Downloading data and Optuna studies from S3

In [9]:
if COLAB: # If you are on Colab, you can use secret environment variables (click on the key in the left panel)
    from google.colab import userdata
    s3 = boto3.client(
        's3',
        aws_access_key_id=userdata.get("ACCESS_KEY"),
        aws_secret_access_key=userdata.get("SECRET_KEY"),
    )
else:
    s3 = boto3.client("s3") # use credentials in the ~/.aws folder

print("list objects in the enefit-competition bucket to check connection :\n")
response = s3.list_objects(
    Bucket='enefit-competition',
    MaxKeys=5,
)
for content in response["Contents"]:
    print(content["Key"])

list objects in the enefit-competition bucket to check connection :

./optuna_studies/lightgbm_preprocessing2_config1.db
configuration_files/lightgbm/config_1.yaml
configuration_files/lightgbm/config_2.yaml
configuration_files/lightgbm/config_3.yaml
configuration_files/lightgbm/config_4.yaml


In [10]:
def download_s3_folder(bucket_name, s3_folder, local_dir):
    """function to download objects from an S3 bucket located in the s3_folder directory"""
    # Ensure the local directory exists
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    # List objects within the specified S3 folder
    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder)

    for page in pages:
        if "Contents" in page:
            for obj in page["Contents"]:
                # Get the path of the object key
                s3_key = obj["Key"]
                if s3_key == s3_folder:
                    continue
                # Remove the prefix from the key to get the relative file path
                relative_path = os.path.relpath(s3_key, s3_folder)

                # Create the full local path
                local_file_path = os.path.join(local_dir, relative_path)

                # Create local directory if not exists
                local_file_dir = os.path.dirname(local_file_path)
                if not os.path.exists(local_file_dir):
                    os.makedirs(local_file_dir)

                # Download the file
                print(f"Downloading s3://{bucket_name}/{s3_key} to {local_file_path}...")
                s3.download_file(bucket_name, s3_key, local_file_path)

In [11]:
bucket_name = "enefit-competition"
s3_folders = ["data/"]  # S3 directories to download from
local_dirs = ["./data/"]  # Local directories to save files

for s3_folder, local_dir in zip(s3_folders, local_dirs):
    download_s3_folder(bucket_name, s3_folder, local_dir)

# Read the data

In [12]:
df = pd.read_csv("./data/consumption.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df.head()

,datetime,prediction_unit_id,consumption
0,2021-09-01,0,96.590
1,2021-09-01,1,17.314
2,2021-09-01,2,656.859
3,2021-09-01,3,59.000
4,2021-09-01,4,501.760


In [13]:
df.shape

(1009176, 3)

# Train/Test split

In [14]:
# taking the last 60 days of each unit for test
df_train, df_test = train_test_split(df, test_window=24 * 60)

In [15]:
assert df.shape[0] == df_train.shape[0] + df_test.shape[0]
assert df.shape[1] == df_train.shape[1] == df_test.shape[1]

In [16]:
test_size = df_test.shape[0] / (df.shape[0])
print(f"test set : {round(test_size*100, 2)}% of the data set")

test set : 9.85% of the data set


# Preprocessing

In [17]:
# get preprocessing function from preprocessing/preprocessing.py
preprocessing = vars()[f"preprocessing_{preprocessing_version}"]

In [18]:
X_train, y_train = preprocessing(df_train)
print(f"X_train shape : {X_train.shape}")
print(f"y_train shape : {y_train.shape}")
X_train.head()

/usr/local/lib/python3.11/dist-packages/mlforecast/core.py:480: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]
/usr/local/lib/python3.11/dist-packages/mlforecast/core.py:480: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]


X_train shape : (854079, 99)
y_train shape : (854079,)


,lag48,lag49,lag50,lag51,lag52,lag53,lag54,lag55,lag56,lag57,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
5795,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,46.840,43.671,...,82.505417,82.612111,81.860500,81.756654,81.308000,81.145600,80.502083,9,5,23
5856,107.129,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,46.840,...,83.241833,83.355714,82.505417,82.612111,81.860500,81.756654,81.308000,9,6,0
5917,81.920,107.129,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,...,83.893958,84.131655,83.241833,83.355714,82.505417,82.612111,81.860500,9,6,1
5978,96.193,81.920,107.129,120.540,134.986,150.412,152.763,136.130,121.033,80.621,...,84.539375,84.841667,83.893958,84.131655,83.241833,83.355714,82.505417,9,6,2
6039,94.536,96.193,81.920,107.129,120.540,134.986,150.412,152.763,136.130,121.033,...,84.552333,85.716806,84.539375,84.841667,83.893958,84.131655,83.241833,9,6,3


In [19]:
X_test, y_test = preprocessing(df_test)
print(f"X_test shape : {X_test.shape}")
print(f"y_test shape : {y_test.shape}")
X_test.head()

X_test shape : (84735, 99)
y_test shape : (84735,)


/usr/local/lib/python3.11/dist-packages/mlforecast/core.py:480: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]
/usr/local/lib/python3.11/dist-packages/mlforecast/core.py:480: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]


,lag48,lag49,lag50,lag51,lag52,lag53,lag54,lag55,lag56,lag57,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
920263,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,246.753,193.661,160.944,...,722.741458,747.325111,723.395542,740.683269,723.535208,734.978880,725.296292,4,2,23
920328,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,246.753,193.661,...,723.696750,753.981429,722.741458,747.325111,723.395542,740.683269,723.535208,4,3,0
920393,999.628,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,246.753,...,724.803458,761.656966,723.696750,753.981429,722.741458,747.325111,723.395542,4,3,1
920458,1001.917,999.628,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,...,726.249667,769.130867,724.803458,761.656966,723.696750,753.981429,722.741458,4,3,2
920523,1014.902,1001.917,999.628,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,...,731.313625,778.516258,726.249667,769.130867,724.803458,761.656966,723.696750,4,3,3


# HPO with Optuna

## Experiment configurations

In [20]:
experiment_config = {
    "int_params": [
        {
            "name": "num_trees",
            "low": 3,
            "high": 10,
            "log": False,
        },
        {
            "name": "max_depth",
            "low": 3,
            "high": 10,
        },
        {
            "name": "num_leaves",
            "low": 5,
            "high": 25,
        },
    ],
    "float_params": [
        {
            "name": "learning_rate",
            "low": 0.001,
            "high": 0.1,
            "log": True,
        },
        {
            "name": "feature_fraction",
            "low": 0.1,
            "high": 0.75,
        }
    ],
    "objective_values": "mean",
}

In [21]:
# Create config files directory if does not exist
path = Path(config_dir_path, model_name)
try:
    path.mkdir(parents=True)
except:
    pass

In [22]:
# Check if the config has already been tested. If not, create a new config file
config_files_path = Path(config_dir_path, model_name)
list_config_files = list(config_files_path.glob("*.yaml"))
n_config_files = len(list_config_files)
if n_config_files == 0:
    config_file = Path(config_files_path, "config_1.yaml")
    with open(config_file, "w") as file:
        yaml.dump(experiment_config, file)

elif config_version:
    config_file = Path(config_files_path, f"config_{config_version}.yaml")

else:
    config_version = 1
    for config_file in natsorted(list_config_files):
        with open(config_file, "rb") as file:
            config = yaml.safe_load(file)
        if experiment_config == config:
            break
        config_version += 1
        if config_version > n_config_files:
            config_file = Path(config_files_path, f"config_{config_version}.yaml")
            with open(config_file, "w") as file:
                yaml.dump(experiment_config, file)

print(f"using {config_file}")

with open(config_file, "rb") as file:
    config = yaml.safe_load(file)

using configuration_files/lightgbm/config_1.yaml


In [23]:
# upload configuration files to S3
list_config_files = list(config_files_path.glob("*.yaml"))
for file_path in list_config_files:
    s3.upload_file(
        file_path,
        bucket_name,
        str(file_path),
    )

## Objective function

In [24]:
def objective(trial):
    # Define hyperparameters
    study_params = {
        "verbosity": -1,
        "random_state": random_state,
        "device": device,
    }
    for int_param in config["int_params"]:
        study_params[int_param["name"]] = trial.suggest_int(**int_param)
    for float_param in config["float_params"]:
        study_params[float_param["name"]] = trial.suggest_float(**float_param)

    # Evaluate model using cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    model = LGBMRegressor(**study_params)
    cv_errors = cross_val_score(
        model, X_train, y_train, scoring="neg_mean_absolute_error", cv=tscv
    )

    # Log CV results
    cv_errors = -cv_errors
    for i in range(len(cv_errors)):
        trial.set_user_attr(f"error_split_{i+1}", cv_errors[i])
    # trial.set_user_attr("cv_errors", list(cv_errors))
    trial.set_user_attr("cv_errors_std", cv_errors.std())

    # Log train MAE
    model.fit(X_train, y_train)
    y_fit = model.predict(X_train)
    train_mae = mean_absolute_error(y_train, y_fit)
    trial.set_user_attr("train_mae", train_mae)

    # Log test MAE
    y_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred)
    trial.set_user_attr("test_mae", test_mae)

    return cv_errors.mean()

## Create and run study

In [25]:
study_name = (
    f"{model_name}_preprocessing{preprocessing_version}_config{config_version}"
)
study_path = f"./optuna_studies/{study_name}.db"
storage_path = "sqlite:///{}".format(study_path)
print(f"Study path : {study_path}")

Study path : ./optuna_studies/lightgbm_preprocessing2_config1.db


In [26]:
# Add stream handler of stdout to show the messages
logger = optuna.logging.get_logger("optuna")
if logger.hasHandlers():
    logger.handlers.clear()
logger.addHandler(logging.StreamHandler(sys.stdout))

# restored_sampler = pickle.load(open("{}-study-sampler.pkl".format(study_name), "rb"))
sampler = optuna.samplers.TPESampler(
    seed=0
)
# For a list a available samplers : https://optuna.readthedocs.io/en/stable/reference/samplers/index.html

In [27]:
# Initialize the Optuna study
study = optuna.create_study(
    study_name=study_name,
    storage=storage_path,
    load_if_exists=True,
    directions=["minimize"],
    sampler=sampler,
    # pruner=pruner,
)

Using an existing study with name 'lightgbm_preprocessing2_config1' instead of creating a new one.


In [28]:
len(study.trials)

63

In [29]:
# !git clone --recursive https://github.com/microsoft/LightGBM

In [30]:
# cd LightGBM

In [31]:
# !cmake -B build -S . -DUSE_GPU=ON
# # if you have installed NVIDIA CUDA to a customized location, you should specify paths to OpenCL headers and library like the following:
# # !cmake -B build -S . -DUSE_GPU=ON -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/
# !cmake --build build -j4

In [32]:
# Execute the hyperparameter optimization trials
%%time

checkpoint = time.time()
for i in range(10):
    study.optimize(objective, n_trials=1)
    if time.time() - checkpoint > 300:
        checkpoint = time.time()
        print("Upload trials database to S3...")
        s3.upload_file(
            study_path,
            bucket_name,
            str(study_path),
        )

print("Final upload of the trials database to S3...")
s3.upload_file(
    study_path,
    bucket_name,
    str(study_path),
)

Trial 63 finished with value: 286.58715438148573 and parameters: {'num_trees': 10, 'max_depth': 10, 'num_leaves': 21, 'learning_rate': 0.08367900688152438, 'feature_fraction': 0.6827975479104864}. Best is trial 45 with value: 249.09055335337084.
Trial 64 finished with value: 254.21363535827618 and parameters: {'num_trees': 10, 'max_depth': 9, 'num_leaves': 17, 'learning_rate': 0.09938219259000007, 'feature_fraction': 0.7200051747102488}. Best is trial 45 with value: 249.09055335337084.
Trial 65 finished with value: 331.69432008567145 and parameters: {'num_trees': 10, 'max_depth': 9, 'num_leaves': 17, 'learning_rate': 0.06818043977872999, 'feature_fraction': 0.5758302215489418}. Best is trial 45 with value: 249.09055335337084.
Trial 66 finished with value: 278.71964859800863 and parameters: {'num_trees': 10, 'max_depth': 8, 'num_leaves': 16, 'learning_rate': 0.0884327350773281, 'feature_fraction': 0.5491091983621129}. Best is trial 45 with value: 249.09055335337084.
Trial 67 finished wi

## Experimental history

In [33]:
study = optuna.create_study(
    study_name=study_name, storage=storage_path, load_if_exists=True
)
print("number of trials in the study :", len(study.trials))
trials_df = study.trials_dataframe().drop(
    columns=["datetime_start", "datetime_complete"]
)
print("best studies :")
trials_df.sort_values(by="value").head(10)

Using an existing study with name 'lightgbm_preprocessing2_config1' instead of creating a new one.
number of trials in the study : 73
best studies :


,number,value,duration,params_feature_fraction,params_learning_rate,params_max_depth,params_num_leaves,params_num_trees,user_attrs_cv_errors_std,user_attrs_error_split_1,user_attrs_error_split_2,user_attrs_error_split_3,user_attrs_error_split_4,user_attrs_error_split_5,user_attrs_test_mae,user_attrs_train_mae,state
45,45,249.090553,0 days 00:00:49.680919,0.592950,0.099724,9,23,10,75.415997,346.039942,210.394874,174.333090,334.207436,180.477424,212.102555,230.847944,COMPLETE
39,39,250.771972,0 days 00:00:51.235220,0.629075,0.099945,8,21,10,77.564305,346.750837,211.175294,173.419901,342.182982,180.330848,213.201423,231.251173,COMPLETE
44,44,250.842708,0 days 00:00:51.208535,0.672526,0.099513,9,23,10,76.807260,347.080723,212.505344,173.999167,339.941160,180.687146,212.194463,231.001628,COMPLETE
14,14,251.448313,0 days 00:00:51.019969,0.729113,0.099795,10,19,10,77.646778,347.489038,210.568447,174.360127,343.242459,181.581493,213.420441,232.360588,COMPLETE
48,48,252.292949,0 days 00:00:51.143024,0.592867,0.097921,8,25,10,76.349221,351.932788,213.709581,177.037104,336.739228,182.046043,214.625587,233.588880,COMPLETE
54,54,254.019986,0 days 00:00:50.666310,0.508833,0.099305,9,19,10,77.389537,349.751750,215.437246,176.411144,345.058526,183.441266,214.660554,233.792348,COMPLETE
64,64,254.213635,0 days 00:00:49.760415,0.720005,0.099382,9,17,10,78.138380,349.639926,214.100961,174.644198,347.543027,185.140065,216.153842,235.010233,COMPLETE
12,12,254.633194,0 days 00:00:50.352679,0.746795,0.098158,10,20,10,79.257412,352.366013,212.516012,176.593988,348.735266,182.954690,216.315961,234.777645,COMPLETE
53,53,258.456998,0 days 00:00:54.001209,0.675689,0.095343,9,22,10,77.921309,361.502741,219.322010,180.965640,343.066167,187.428432,220.664927,240.518175,COMPLETE
36,36,260.481677,0 days 00:00:52.232955,0.747094,0.094849,10,21,10,79.836587,362.662270,219.089831,181.572617,351.164231,187.919433,221.130396,241.367241,COMPLETE
